pip install git+https://github.com/asimov-academy/Asimov_TV_connector.git

https://python.plainenglish.io/extracting-data-from-trading-view-253919ba7194

In [34]:
from tvDatafeed import TvDatafeed, Interval
from datetime import datetime, timedelta
import pandas as pd

username = open('login_tv.txt', 'r').read()
password = open('passwd_tv.txt', 'r').read()

tv = TvDatafeed(username, password)

In [23]:
# # tv.search_symbol('PETR4','BMFBOVESPA')
# df = tv.get_hist(symbol='PETR4',exchange='BMFBOVESPA',interval=Interval.in_5_minute, n_bars= 5000)
# df.head(3)

In [24]:
df = tv.get_hist(symbol='CAML3',exchange='BMFBOVESPA',interval=Interval.in_1_minute, n_bars= 1)
df.tail(50)

,symbol,open,high,low,close,volume
datetime,,,,,,
2026-01-02 17:49:00,BMFBOVESPA:CAML3,5.53,5.55,5.5,5.5,1200.0


In [35]:
acoes = [
    "DIRR3",
    "BRAP4",
    "POMO4",
    "LOGG3",
    "CMIG4",
    "CYRE3",
    "SHUL4",
    "CSUD3",
    "ITSA3",
    "TRIS3"
]


In [36]:
nbars=5
for idx,row in enumerate(acoes):
    i=row
    if idx == 0:       
        df =  tv.get_hist(symbol= i,exchange='BMFBOVESPA', n_bars= nbars)
        df.rename(columns={ 'open':i+'_open' , 'high' : i+'_high' , 
                       'low' : i+'_low' , 'close' : i+'_close',
                       'volume' : i+'_volume'}
                       ,inplace=True)
        del df['symbol']
    else:
        df1 =  tv.get_hist(symbol= i,exchange='BMFBOVESPA', n_bars= nbars)
        df1.rename(columns={ 'open':i+'_open' , 'high' : i+'_high' , 
                       'low' : i+'_low' , 'close' : i+'_close',
                       'volume' : i+'_volume'}
                       ,inplace=True)
        del df1['symbol']

        df = pd.merge(df,df1,how='inner',left_index=True,right_index=True)

df.tail()

,DIRR3_open,DIRR3_high,DIRR3_low,DIRR3_close,DIRR3_volume,BRAP4_open,BRAP4_high,BRAP4_low,BRAP4_close,BRAP4_volume,...,ITSA3_open,ITSA3_high,ITSA3_low,ITSA3_close,ITSA3_volume,TRIS3_open,TRIS3_high,TRIS3_low,TRIS3_close,TRIS3_volume
datetime,,,,,,,,,,,,,,,,,,,,,
2025-12-23 10:00:00,13.51,14.01,13.44,14.01,8781900.0,20.59,20.85,20.30,20.30,9048200.0,...,11.77,11.90,11.70,11.80,101500.0,6.42,6.62,6.42,6.59,371000.0
2025-12-26 10:00:00,13.94,14.07,13.77,14.03,3659600.0,20.30,20.49,20.25,20.49,2029300.0,...,11.92,11.94,11.73,11.94,121100.0,6.59,6.62,6.45,6.50,519600.0
2025-12-29 10:00:00,14.00,14.01,13.67,13.83,7662300.0,20.59,20.61,19.96,20.00,2676200.0,...,11.87,11.95,11.80,11.90,182000.0,6.18,6.28,6.01,6.12,680700.0
2025-12-30 10:00:00,13.88,14.27,13.84,14.12,8914300.0,20.11,20.22,19.89,19.90,2983900.0,...,11.96,12.10,11.95,11.99,1159700.0,6.12,6.30,6.11,6.26,258300.0
2026-01-02 10:00:00,14.15,14.23,13.52,13.63,10710200.0,19.90,20.08,19.86,20.08,2851300.0,...,12.00,12.18,11.97,12.11,206100.0,6.30,6.34,6.10,6.15,479900.0


In [39]:

# # 1. Transpor preservando índice
# df2 = df.reset_index().T

# # 2. Garantir comportamento futuro: inferir tipos
# df2.index = df2.index.infer_objects()
# df2.columns = df2.iloc[0]
# df2 = df2[1:]

# 3. Função segura para transformar decimais em vírgula
def convert_decimal(x):
    try:
        f = float(x)
        return str(f).replace('.', ',')
    except:
        return x

# 4. Usar .map() em cada coluna (substitui applymap sem warnings)
df = df.map(convert_decimal)

# 5. Salvar
df.to_csv("data/position.csv", sep=';', index=True)

In [40]:
df

,DIRR3_open,DIRR3_high,DIRR3_low,DIRR3_close,DIRR3_volume,BRAP4_open,BRAP4_high,BRAP4_low,BRAP4_close,BRAP4_volume,...,ITSA3_open,ITSA3_high,ITSA3_low,ITSA3_close,ITSA3_volume,TRIS3_open,TRIS3_high,TRIS3_low,TRIS3_close,TRIS3_volume
datetime,,,,,,,,,,,,,,,,,,,,,
2025-12-23 10:00:00,"13,51","14,01","13,44","14,01","8781900,0","20,59","20,85","20,3","20,3","9048200,0",...,"11,77","11,9","11,7","11,8","101500,0","6,42","6,62","6,42","6,59","371000,0"
2025-12-26 10:00:00,"13,94","14,07","13,77","14,03","3659600,0","20,3","20,49","20,25","20,49","2029300,0",...,"11,92","11,94","11,73","11,94","121100,0","6,59","6,62","6,45","6,5","519600,0"
2025-12-29 10:00:00,"14,0","14,01","13,67","13,83","7662300,0","20,59","20,61","19,96","20,0","2676200,0",...,"11,87","11,95","11,8","11,9","182000,0","6,18","6,28","6,01","6,12","680700,0"
2025-12-30 10:00:00,"13,88","14,27","13,84","14,12","8914300,0","20,11","20,22","19,89","19,9","2983900,0",...,"11,96","12,1","11,95","11,99","1159700,0","6,12","6,3","6,11","6,26","258300,0"
2026-01-02 10:00:00,"14,15","14,23","13,52","13,63","10710200,0","19,9","20,08","19,86","20,08","2851300,0",...,"12,0","12,18","11,97","12,11","206100,0","6,3","6,34","6,1","6,15","479900,0"


In [28]:
url0 = 'https://raw.githubusercontent.com/BDonadelli/Finance-playground/refs/heads/main/data/'

url  = {'ibra50' :url0+'Cart_IBr50.csv',
        'ibra100':url0+'Cart_IBr100.csv',
        'idiv'   :url0+'Cart_Idiv.csv',
        'ibov'   :url0+'Cart_Ibov.csv',
        'small'  :url0+'Cart_Small.csv'
}
indice = pd.read_csv(url['ibov'] ,
                     sep=';' , decimal=',' , thousands='.' ,
                     skiprows=1 , skipfooter=2 , encoding='latin-1',
                     index_col=False , engine='python')

indice

,Código,Ação,Tipo,Qtde. Teórica,Part. (%)
0,ALOS3,ALLOS,ON NM,476976044,0.500
1,ABEV3,AMBEV S/A,ON,4394835131,2.724
2,ASAI3,ASSAI,ON NM,1345897506,0.636
3,AURE3,AUREN,ON NM,323738747,0.145
4,AZZA3,AZZAS 2154,ON NM,136643320,0.257
...,...,...,...,...,...
79,VAMO3,VAMOS,ON NM,485166826,0.088
80,VBBR3,VIBRA,ON NM,1113939036,1.154
81,VIVA3,VIVARA S.A.,ON NM,123160591,0.154
82,WEGE3,WEG,ON NM,1482105837,2.922


In [29]:
lista = ['PETR3','PETR4','VALE3','BBAS3']

In [30]:
for idx,row in enumerate(lista):
    i=row
    if idx == 0:       
        df =  tv.get_hist(symbol= i,exchange='BMFBOVESPA', n_bars= 100)
        df.rename(columns={ 'open':i+'_open' , 'high' : i+'_high' , 
                       'low' : i+'_low' , 'close' : i+'_close',
                       'volume' : i+'_volume'},inplace=True)
        del df['symbol']
    else:
        df1 =  tv.get_hist(symbol= i,exchange='BMFBOVESPA', n_bars= 100)
        df1.rename(columns={ 'open':i+'_open' , 'high' : i+'_high' , 
                       'low' : i+'_low' , 'close' : i+'_close',
                       'volume' : i+'_volume'},inplace=True)
        del df1['symbol']

        df = pd.merge(df,df1,how='inner',left_index=True,right_index=True)

df.to_csv("data/tvohlc.csv" , sep=';' , decimal='.')

In [31]:
def import_data(csv): 
    """ Renomeando as colunas para aritmetica com dataframe """
    def extrair_caracteres(s):
        import re
        pattern = re.compile(r"([^_]+)_")
    
        match = pattern.match(s)
        if match:
                return match.group(1)
        else:
             return None
    
    df = pd.read_csv(csv, sep=';' , decimal='.')
    df.index = pd.to_datetime(df['datetime'])#, format = "%y-%m-%d")
    df = df.drop('datetime', axis=1)
    
    close_columns    = []
    high_columns     = []
    low_columns      = []
    open_columns     = []
    volume_columns   = []

    for i in df.columns:
        if "close" in i:
            close_columns.append(i)
        elif "high" in i:
            high_columns.append(i)
        elif "low" in i:
            low_columns.append(i)
        elif "open_int" in i:
            open_int_columns.append(i)
        elif "open" in i:
            open_columns.append(i)
        elif "volume" in i:
            volume_columns.append(i)

    dicH = {s:extrair_caracteres(s) for s in high_columns}
    dicL = {s:extrair_caracteres(s) for s in low_columns}
    dicO = {s:extrair_caracteres(s) for s in open_columns}
    dicC = {s:extrair_caracteres(s) for s in close_columns}
    dicV = {s:extrair_caracteres(s) for s in volume_columns}
    
    close_df    = df[close_columns].rename(columns=dicC)
    high_df     = df[high_columns].rename(columns=dicH)
    low_df      = df[low_columns].rename(columns=dicL)
    open_df     = df[open_columns].rename(columns=dicO)
    volume_df   = df[volume_columns].rename(columns=dicV)
 
    return [close_df, high_df, low_df, open_df, volume_df]